In [1]:
from threading import Thread
import time
import os

In [2]:
import rospy
import actionlib
from assignment_2_2022.msg import PlanningAction, PlanningGoal
from unige_rt1_assignment2.srv import GoalInfo
from unige_rt1_assignment2.msg import RoboStatusMsg
from sensor_msgs.msg import LaserScan

In [4]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from IPython.core.display import HTML
from ipywidgets import Layout

In [6]:
class RoboClient:
    def __init__(self):
        rospy.init_node('robo_client', anonymous=True)
        
        # Service client to get the goals info
        self.goal_info_client = rospy.ServiceProxy("/goals_service", GoalInfo)

        # Dictionary to store the robot position
        self. robo_position = {
            "x": 0.0,
            "y": 0.0
        }
        # Subscriber to get the robot position
        self.robo_position_sub = rospy.Subscriber("/robot/robo_stats", RoboStatusMsg, self.robo_position_callback)

        # variable to store the closest obstacle, defined as infinity at the beginning
        self.closest_obstacle = float("inf")
        # Subscriber to get the laser scan information for distance to obstacles
        self.laser_scan_sub = rospy.Subscriber("/scan", LaserScan, self.laser_scan_callback)

        # Action client to send the goals
        self.target_ac = actionlib.SimpleActionClient("/reaching_goal", PlanningAction)
        rospy.loginfo("Waiting for action server to start.")
        self.target_ac.wait_for_server()
        rospy.loginfo("Action server started!")

    def send_goal(self, x, y):
        goal = PlanningGoal()
        goal.target_pose.pose.position.x = x
        goal.target_pose.pose.position.y = y
        self.target_ac.send_goal(goal)

        if rospy.has_param("/robot/goal_pos_param"):
            rospy.set_param("/robot/goal_pos_param/x_goal", x)
            rospy.set_param("/robot/goal_pos_param/y_goal", y)
        else:
            rospy.logerr("Goal position parameter not found on the parameter server.")

    def cancel_goal(self):
        if self.target_ac.get_state() == actionlib.GoalStatus.ACTIVE:
            print("Cancelling current goal...")
            self.target_ac.cancel_goal()
        else:
            print("No goal is currently executing.")
    
    def get_goal_info(self):
        try:
            goals_info_response = self.goal_info_client()
            return goals_info_response.msg_feedback
        except rospy.ServiceException as e:
            print("Service call failed: %s"%e)
            return None

    def robo_position_callback(self, msg):
        self.robo_position["x"] = msg.pos_x
        self.robo_position["y"] = msg.pos_y
    
    def get_robo_position(self):
        return self.robo_position["x"], self.robo_position["y"]

    def laser_scan_callback(self, msg):
        self.closest_obstacle = min(msg.ranges)

    
    def get_closest_obstacle(self):
        return self.closest_obstacle

    def spin(self):
        rospy.spin()

In [7]:
robo_client = RoboClient()

def on_send_button_click(button):
    x = float(x_text.value)
    y = float(y_text.value)
    robo_client.send_goal(x, y)
    print(f"Sending new goal: ({x}, {y})")

def on_cancel_button_click(button):
    robo_client.cancel_goal()

x_text = widgets.FloatText(description="X:")
y_text = widgets.FloatText(description="Y:")

send_button = widgets.Button(description="Send Goal")
send_button.on_click(on_send_button_click)

cancel_button = widgets.Button(description="Cancel Goal")
cancel_button.on_click(on_cancel_button_click)

goal_ui = widgets.HBox([x_text, y_text, send_button, cancel_button])

[INFO] [1683392672.564516, 1625.492000]: Waiting for action server to start.
[INFO] [1683392672.589452, 1625.517000]: Action server started!


In [8]:
# Information Display UI

# Widget to display the robot position
robo_pos_widget_label = widgets.HTML(value="<p style='font-size: 14px;'><b>Robot position:</b> N/A</p>")

# Widget to display the closest obstacle
closest_obstacle_widget_label = widgets.HTML(value="<p style='font-size: 14px;'><b>Distance to closest obstacle:</b> N/A</p>")

# Jupyter widget for goals reached and cancelled
goal_info_widget_label = widgets.HTML(value="<p style='font-size: 14px;'><b>Goals reached:</b> N/A</p>")

# Combine the labels into a single UI using a VBox
info_ui = widgets.VBox([robo_pos_widget_label, closest_obstacle_widget_label, goal_info_widget_label])

In [9]:
# heading
heading = widgets.HTML(value="<center><h2><u>Robot Controller Interface</u></h2></center>")

# Combine the goal UI and the information UI into a single UI using a VBox
combined_ui = widgets.VBox([heading, goal_ui, info_ui])

# Display the combined UI
display(combined_ui)

In [11]:
def updated_robo_position():
    while not rospy.is_shutdown():
        robo_x_pos, robo_y_pos = robo_client.get_robo_position()
        robo_pos_widget_label.value = f"Robot position: ({robo_x_pos:.2f}, {robo_y_pos:.2f})"

        closest_obstacle_dist = robo_client.get_closest_obstacle()
        closest_obstacle_widget_label.value = f"Distance to closest obstacle: {closest_obstacle_dist:.2f}"

        rospy.sleep(0.75)

position_thread = Thread(target=updated_robo_position)
position_thread.start()

In [12]:
def update_goal_info():
    while not rospy.is_shutdown():
        goal_info = robo_client.get_goal_info()
        if goal_info is not None:
            goal_info_widget_label.value = f"Goals Information: {goal_info}"
        rospy.sleep(0.75)

goal_info_thread = Thread(target=update_goal_info)
goal_info_thread.start()

In [3]:
# from jupyros.ros1 import ros3d

In [5]:
# # %matplotlib notebook
# import numpy as np
# import matplotlib.pyplot as plt

In [10]:
# laserscan_output = widgets.Output()
# display(laserscan_output)

In [13]:
# def plot_robo_pos(robo_cli):
#     fig, ax = plt.subplots()

#     while not rospy.is_shutdown():
#         clear_output(wait=True) # Clear the previous graph
#         robo_x_pos, robo_y_pos = robo_cli.get_robo_position()

#         ax.set_xlim(robo_x_pos - 10, robo_x_pos + 10)
#         ax.set_ylim(robo_y_pos - 10, robo_y_pos + 10)

#         ax.plot(robo_x_pos, robo_y_pos, "bo", markersize=10, label="Robot Position")
#         ax.legend()

#         plt.grid(True)
#         plt.xlabel("X")
#         plt.ylabel("Y")
#         plt.title("Robot Position")

#         plt.pause(0.1)
    
#     plt.close(fig)

In [22]:
# %matplotlib notebook

# import matplotlib.pyplot as plt

# def plot_robot_position(client):
#     fig, ax = plt.subplots()
#     robot_position_plot, = ax.plot([], [], 'bo', markersize=10, label='Robot Position')
#     ax.legend()
#     plt.grid(True)
#     plt.xlabel("X-axis")
#     plt.ylabel("Y-axis")
#     plt.title("Real-time Robot Position")

#     while not rospy.is_shutdown():
#         robo_x_pos, robo_y_pos = client.get_robo_position()

#         # Set axis limits
#         ax.set_xlim(robo_x_pos - 10, robo_x_pos + 10)
#         ax.set_ylim(robo_y_pos - 10, robo_y_pos + 10)

#         # Update the robot position
#         robot_position_plot.set_data(robo_x_pos, robo_y_pos)

#         fig.canvas.draw()
#         fig.canvas.flush_events()
#         time.sleep(0.1)

#     plt.close(fig)

# plot_position_thread = Thread(target=plot_robot_position, args=(robo_client,))
# plot_position_thread.start()


In [30]:
# v = ros3d.Viewer()
# v.layout = Layout(border="3px solid red", width="200px", height="50px")
# rc = ros3d.ROSConnection(url="ws://localhost:9090")
# tf_client = ros3d.TFClient(ros=rc, fixed_frame='')

# laser_view = ros3d.LaserScan(topic="/scan", ros=rc, tf_client=tf_client)
# g = ros3d.GridModel()
# v.objects = [g, laser_view]

# v